In [1]:
! pip install SpeechRecognition pyaudio pydub
! pip install torch

In [2]:
!pip install sounddevice
!pip install scipy
!pip install transformers
!pip install librosa
!pip install torch   

In [17]:
import os
import torch
import sounddevice as sd
from scipy.io.wavfile import write
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import librosa

# Use current working directory to save/load model
SCRIPT_DIR = os.getcwd()  # Get current working directory
MODEL_PATH = os.path.join(SCRIPT_DIR, "wav2vec2_model")  # Path to save the model

# Create the directory if it doesn't exist
os.makedirs(MODEL_PATH, exist_ok=True)

# Load or download model to current directory
def load_or_download_model():
    # Download and save model locally if not already saved
    if not os.path.exists(os.path.join(MODEL_PATH, "pytorch_model.bin")):
        print("🌐 Downloading model from Hugging Face...")
        
        # Load the model and processor from Hugging Face
        processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
        model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
        
        # Save the model and processor locally as .bin and other necessary files
        processor.save_pretrained(MODEL_PATH)  # Save processor files
        model.save_pretrained(MODEL_PATH)  # Save model weights (pytorch_model.bin) and config
        
        print(f"💾 Model saved locally to: {MODEL_PATH}")
    else:
        print(f"📦 Loading model from local directory: {MODEL_PATH}")
        processor = Wav2Vec2Processor.from_pretrained(MODEL_PATH)
        model = Wav2Vec2ForCTC.from_pretrained(MODEL_PATH)

    return processor, model

# Record audio from mic
def record_audio(duration=5, filename="mic_audio.wav"):
    print("🎤 Recording...")
    fs = 16000  # sample rate
    audio = sd.rec(int(duration * fs), samplerate=fs, channels=1)
    sd.wait()  # wait until recording is finished
    write(filename, fs, audio)
    print(f"✅ Saved: {filename}")
    return filename

# Transcribe using Wav2Vec2
def transcribe_wav2vec2(file_path):
    processor, model = load_or_download_model()

    audio, _ = librosa.load(file_path, sr=16000)
    input_values = processor(audio, return_tensors="pt", sampling_rate=16000).input_values

    with torch.no_grad():
        logits = model(input_values).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.decode(predicted_ids[0])
    print("📝 Transcription:", transcription)

# ---------- Run ----------
# Uncomment below lines to run the code

# file = record_audio()
# transcribe_wav2vec2(file)
load_or_download_model()

🌐 Downloading model from Hugging Face...


Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You sho

💾 Model saved locally to: d:\ai\wav2vec2_model


(Wav2Vec2Processor:
 - feature_extractor: Wav2Vec2FeatureExtractor {
   "do_normalize": true,
   "feature_extractor_type": "Wav2Vec2FeatureExtractor",
   "feature_size": 1,
   "padding_side": "right",
   "padding_value": 0.0,
   "processor_class": "Wav2Vec2Processor",
   "return_attention_mask": false,
   "sampling_rate": 16000
 }
 
 - tokenizer: Wav2Vec2CTCTokenizer(name_or_path='facebook/wav2vec2-base-960h', vocab_size=32, model_max_length=9223372036854775807, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=True),
 Wav2Vec2ForCTC(
   (wav2vec2): Wav2Vec2Model(
     (feature_extractor): Wav2Vec2FeatureEncoder(
       (conv_layers): ModuleList(
         (0): Wav2Vec2GroupNormConvLayer(
           (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
           (activation): GELUActivation()
           (layer_norm): GroupNorm(512, 